# Classification-of-CGM-Timeseries-data-into-Meal-or-No-Meal-Data

In [32]:
import pandas as pd
import csv
import numpy as np
from collections import Counter 
import warnings
warnings.filterwarnings('ignore')
import io

In [33]:
from scipy.fftpack import fft, ifft
def fft_and_peaks(df):
        
    fft_val=[]
    fft_val.append(abs(fft(df)))
        
    peak_val=[]
    for z in range(len(fft_val)):
        a = list(set(fft_val[z]))
        a.sort()
        a = a[::-1][1:5]
        peak_val.append(a)
    return(fft_val,peak_val)

In [34]:
def velocity(data):
    
    data = data.reset_index().drop(columns = 'index').T
    mean = []
    std =[]
    median =[]
    
    interval = 10
    for k in range(len(data)):
        window_size = 5
        velocity = []
        row_data = data.iloc[k].values
        row_length = len(row_data)
        counter = 0
        cgmvel = []
        for i in range((len(row_data) - window_size)):
            cgmvel.append(counter)
            counter += 5
            p = (row_data[i] - row_data[i + window_size])
            vel = p / interval
            velocity.append(vel)
        mean.append(np.mean(velocity))
        std.append(np.std(velocity))
        median.append(np.median(velocity))
        
    df = list(zip(mean, std, median))[0]
    return df

In [35]:
def autocorrelation(df):
        
    autocorr_lag_2=[]
    autocorr_lag_3=[]
    autocorr_lag_4=[]
    autocorr_lag_5=[]
    autocorr_lag_6=[]
    autocorr_lag_7=[]
    autocorr_lag_8=[]
    autocorr_lag_9=[]
    autocorr_lag_10=[]
    autocorr_lag_11=[]
    autocorr_lag_12=[]
    autocorr_lag_13=[]
    autocorr_lag_14=[]

    auto_corr_2= df.autocorr(lag=2)
    autocorr_lag_2.append(auto_corr_2)

    auto_corr_3= df.autocorr(lag=3)
    autocorr_lag_3.append(auto_corr_3)

    auto_corr_4= df.autocorr(lag=4)
    autocorr_lag_4.append(auto_corr_4)

    auto_corr_5= df.autocorr(lag=5)
    autocorr_lag_5.append(auto_corr_5)

    auto_corr_6= df.autocorr(lag=6)
    autocorr_lag_6.append(auto_corr_6)

    auto_corr_7= df.autocorr(lag=7)
    autocorr_lag_7.append(auto_corr_7)

    auto_corr_8= df.autocorr(lag=8)
    autocorr_lag_8.append(auto_corr_8)

    auto_corr_9= df.autocorr(lag=9)
    autocorr_lag_9.append(auto_corr_9)

    auto_corr_10=df.autocorr(lag=10)
    autocorr_lag_10.append(auto_corr_10)

    auto_corr_11= df.autocorr(lag=11)
    autocorr_lag_11.append(auto_corr_11)

    auto_corr_12= df.autocorr(lag=12)
    autocorr_lag_12.append(auto_corr_12)

    auto_corr_13= df.autocorr(lag=13)
    autocorr_lag_13.append(auto_corr_13)

    auto_corr_14= df.autocorr(lag=14)
    autocorr_lag_14.append(auto_corr_14)

    df1 = list(zip(autocorr_lag_2, autocorr_lag_3,autocorr_lag_4,autocorr_lag_5,autocorr_lag_6,autocorr_lag_7,autocorr_lag_8,autocorr_lag_9,autocorr_lag_10,autocorr_lag_11,autocorr_lag_12,autocorr_lag_13,autocorr_lag_14))[0]
    return (df1)

In [36]:
def polyfit_coeffs(df):
    coeff_list = []
    y = df.to_numpy()
    time = np.linspace(1,df.shape[0],df.shape[0])
    coeff = np.polyfit(time,y,6)
    coeff_list.append(coeff)
    df1 = coeff_list[0]
    return df1

In [37]:
def psd(df):
    import scipy.signal as signal
    max_amp=[]
    std_amp=[]
    mean_amp=[]
    
    df = df.reset_index().drop(columns = 'index').T
    
    for k in range(len(df)):
        v, welch_values  = np.array((signal.welch(df.iloc[k].values)))
        max_amp.append(np.sqrt(max(welch_values)))
        std_amp.append(np.std(np.sqrt(welch_values)))
        mean_amp.append(np.mean(np.sqrt(welch_values)))
        
    df1 = list(zip(max_amp, std_amp, mean_amp))[0]
    return df1
   

In [38]:
def file_read(filename):
    f = open(filename)
    csv_f = csv.reader(f)

    data = []

    for row in csv_f:
        data.append(row)
    return data

In [39]:
def calculate_features(data):   
    fft_peak_val =[]
    cgm_vel = []
    auto_corr=[]
    polyfit_coeff=[]
    psd_val = []


    for i in range(len(data)):
        intermediate = pd.DataFrame(data[i]).reset_index().drop(columns = 'index')
        each_row = pd.to_numeric(intermediate[0],errors='coerce').interpolate().dropna()

        if (len(each_row)!=0):
            x,y = fft_and_peaks(each_row)
            fft_peak_val.append(y[0])

            v = velocity(each_row)
            cgm_vel.append(v)

            acr = autocorrelation(each_row)
            auto_corr.append(acr)

            p = polyfit_coeffs(each_row)
            polyfit_coeff.append(p)

            psd1 = psd(each_row)
            psd_val.append(psd1)
        else:
            pass
    
    peak_val = pd.DataFrame(list(fft_peak_val),columns = ['Peak 2','Peak 3','Peak 4','Peak 5'])
    cgm_vel_val = pd.DataFrame(cgm_vel,columns=['vel_Mean','vel_STD','vel_Median'])
    autocorr_val = pd.DataFrame(auto_corr,columns=['autocorr_lag_2', 'autocorr_lag_3','autocorr_lag_4','autocorr_lag_5','autocorr_lag_6','autocorr_lag_7','autocorr_lag_8','autocorr_lag_9','autocorr_lag_10','autocorr_lag_11','autocorr_lag_12','autocorr_lag_13','autocorr_lag_14']) 
    polyfit_val = pd.DataFrame(polyfit_coeff,columns=['coeff1','coeff2','coeff3','coeff4','coeff5','coeff6','coeff7'])
    psd_val = pd.DataFrame(psd_val,columns =['max amplitude', 'std amplitude','mean amplitude'])
    
    dataset = pd.concat([peak_val,cgm_vel_val,autocorr_val,polyfit_val,psd_val],axis=1).fillna(0)
    return(dataset)

# Reading Meal Data and Calculating the features

In [40]:
meal_pat1 = file_read('mealData1.csv')
features_meal_pat1 = calculate_features(meal_pat1)

In [41]:
meal_pat2 = file_read('mealData2.csv')
features_meal_pat2 = calculate_features(meal_pat2)

In [42]:
meal_pat3 = file_read('mealData3.csv')
features_meal_pat3 = calculate_features(meal_pat3)

In [43]:
meal_pat4 = file_read('mealData4.csv')
features_meal_pat4 = calculate_features(meal_pat4)

In [44]:
meal_pat5 = file_read('mealData5.csv')
features_meal_pat5 = calculate_features(meal_pat5)

In [45]:
meal_data = pd.concat([features_meal_pat1,features_meal_pat2,features_meal_pat3,features_meal_pat4,features_meal_pat5],axis=0)

In [46]:
meal_data['Label']=1

# Reading Non-Meal Data and Calculating the features

In [47]:
nomeal_pat1 = file_read('Nomeal1.csv')
features_nomeal_pat1 = calculate_features(nomeal_pat1)

In [48]:
nomeal_pat2 = file_read('Nomeal2.csv')
features_nomeal_pat2 = calculate_features(nomeal_pat2)

In [49]:
nomeal_pat3 = file_read('Nomeal3.csv')
features_nomeal_pat3 = calculate_features(nomeal_pat3)

In [50]:
nomeal_pat4 = file_read('Nomeal4.csv')
features_nomeal_pat4= calculate_features(nomeal_pat2)

In [51]:
nomeal_pat5 = file_read('Nomeal5.csv')
features_nomeal_pat5 = calculate_features(nomeal_pat5)

In [52]:
nomeal_data = pd.concat([features_nomeal_pat1,features_nomeal_pat2,features_nomeal_pat3,features_nomeal_pat4,features_nomeal_pat5],axis=0)

In [53]:
nomeal_data['Label']=0

# Combining Meal and No Meal Data

In [54]:
from sklearn.utils import shuffle
dataset = shuffle(pd.concat([meal_data,nomeal_data],axis=0).fillna(0)).reset_index().drop(columns = ['index'])

# PCA

In [55]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats
feature_matrix = StandardScaler().fit_transform(dataset.drop(columns= ['Label']))
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(feature_matrix)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5'])

# K-Fold Cross Validation

In [56]:
principalDf['Label'] = dataset['Label']

In [57]:
principalDfData = principalDf.drop(columns = ['Label'])

In [58]:
def get_score(model, X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    return model.score(X_test,y_test)

In [59]:
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
scores_rf = []

from sklearn.model_selection import KFold
kfold = KFold(n_splits=10,shuffle=False)
for train_index, test_index in kfold.split(principalDfData):
    X_train,X_test,y_train,y_test = principalDfData.loc[train_index],principalDfData.loc[test_index],\
    principalDf.Label.loc[train_index],principalDf.Label.loc[test_index]
    scores_rf.append(get_score(RandomForestClassifier(criterion="entropy"), X_train, X_test, y_train, y_test))

In [60]:
print(np.mean(scores_rf))

0.6979591836734693


# Pickling the Model

In [61]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
X, y= principalDfData, principalDf['Label']
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [62]:
from joblib import dump, load
dump(clf, 'RandomForestClassifier.pickle')

['RandomForestClassifier.pickle']